In [50]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pandas import json_normalize
import datetime 

In [37]:
# Get response from ApeWisdom, Wall Street Bets 

# trending stocks from the last 24 hours

wall_street_bets_response = requests.get("https://apewisdom.io/api/v1.0/filter/wallstreetbets").json()["results"]

wall_street_bets_response[:5]

[{'rank': 1,
  'ticker': 'SPY',
  'name': 'SPDR S&amp;P 500 ETF Trust',
  'mentions': '513',
  'upvotes': '3094',
  'rank_24h_ago': '1',
  'mentions_24h_ago': '526'},
 {'rank': 2,
  'ticker': 'GME',
  'name': 'GameStop',
  'mentions': '266',
  'upvotes': '2871',
  'rank_24h_ago': '3',
  'mentions_24h_ago': '164'},
 {'rank': 3,
  'ticker': 'TSLA',
  'name': 'Tesla',
  'mentions': '170',
  'upvotes': '463',
  'rank_24h_ago': '2',
  'mentions_24h_ago': '250'},
 {'rank': 4,
  'ticker': 'AMD',
  'name': 'AMD',
  'mentions': '145',
  'upvotes': '474',
  'rank_24h_ago': '7',
  'mentions_24h_ago': '76'},
 {'rank': 5,
  'ticker': 'WMT',
  'name': 'Walmart',
  'mentions': '133',
  'upvotes': '564',
  'rank_24h_ago': '22',
  'mentions_24h_ago': '22'}]

In [43]:
# Most discussed Stocks on r/wallstreetbets in the past 24 hours

wall_street_bets = json_normalize(wall_street_bets_response)

wall_street_bets

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,SPY,SPDR S&amp;P 500 ETF Trust,513,3094,1,526
1,2,GME,GameStop,266,2871,3,164
2,3,TSLA,Tesla,170,463,2,250
3,4,AMD,AMD,145,474,7,76
4,5,WMT,Walmart,133,564,22,22
...,...,...,...,...,...,...,...
95,96,HE,Hawaiian Electric Industries,7,24,582,1
96,97,HAS,Hasbro,7,25,66,7
97,98,CS,Credit Suisse,7,20,149,3
98,99,REAL,The RealReal,7,28,114,5


In [45]:
# Top 5 discussed Stocks on r/wallstreetbets in the past 24 hours

top_5_trending_stocks = list(wall_street_bets.sort_values("rank")[:5]["ticker"])

top_5_trending_stocks

['SPY', 'GME', 'TSLA', 'AMD', 'WMT']

In [49]:
# Use this list to call Alpaca API to get Price, within a timeframe (prior 48 hours)

load_dotenv()

alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version ="v2")

In [89]:
# Set parameters for alpaca api call

today = datetime.datetime.today()
delta = datetime.timedelta(days = 2)
start_date = (today - delta).strftime("%Y-%m-%d")
timeframe ='30Min'

In [120]:
prices_df = alpaca.get_bars(top_5_trending_stocks, timeframe = timeframe, start = start_date).df

prices_df = prices_df.loc[prices_df['symbol'].isin(top_5_trending_stocks)][["close", "symbol"]]

stock_prices_top5 = prices_df.pivot_table(columns="symbol", index="timestamp")

stock_prices_top5

close                                    
symbol                        AMD       GME     SPY    TSLA       WMT
timestamp                                                            
2022-05-16 08:00:00+00:00   94.29   98.8900  400.30  762.66       NaN
2022-05-16 08:30:00+00:00   94.73   99.5000  401.13  768.31       NaN
2022-05-16 09:00:00+00:00   94.49   99.5000  400.28  768.87       NaN
2022-05-16 09:30:00+00:00   94.00   98.6100  399.72  767.03  148.1400
2022-05-16 10:00:00+00:00   94.54   98.1500  400.40  772.00  148.1300
...                           ...       ...     ...     ...       ...
2022-05-17 21:30:00+00:00  102.69  100.5101  408.67  761.75  131.7500
2022-05-17 22:00:00+00:00  102.90  100.6300  408.98  762.00  131.5000
2022-05-17 22:30:00+00:00  102.57  101.0000  408.67  762.40  131.4300
2022-05-17 23:00:00+00:00  102.55  101.3900  408.59  763.30  131.4999
2022-05-17 23:30:00+00:00  102.80  101.3200  408.19  761.05  131.4900

[64 rows x 5 columns]

In [95]:
crypto_response = requests.get("https://apewisdom.io/api/v1.0/filter/cryptocurrency").json()["results"]
crypto_response[:5]

[{'rank': 1,
  'ticker': 'BTC.X',
  'name': 'Bitcoin',
  'mentions': '667',
  'upvotes': '2000',
  'rank_24h_ago': '1',
  'mentions_24h_ago': '771'},
 {'rank': 2,
  'ticker': 'ETH.X',
  'name': 'Ethereum',
  'mentions': '313',
  'upvotes': '875',
  'rank_24h_ago': '2',
  'mentions_24h_ago': '433'},
 {'rank': 3,
  'ticker': 'UST.X',
  'name': 'TerraUSD',
  'mentions': '252',
  'upvotes': '1539',
  'rank_24h_ago': '3',
  'mentions_24h_ago': '445'},
 {'rank': 4,
  'ticker': 'LUNA.X',
  'name': 'Terra',
  'mentions': '213',
  'upvotes': '790',
  'rank_24h_ago': '4',
  'mentions_24h_ago': '394'},
 {'rank': 5,
  'ticker': 'USDT.X',
  'name': 'Tether',
  'mentions': '139',
  'upvotes': '711',
  'rank_24h_ago': '5',
  'mentions_24h_ago': '232'}]

In [97]:
crypto = json_normalize(crypto_response)
crypto

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,BTC.X,Bitcoin,667,2000,1,771
1,2,ETH.X,Ethereum,313,875,2,433
2,3,UST.X,TerraUSD,252,1539,3,445
3,4,LUNA.X,Terra,213,790,4,394
4,5,USDT.X,Tether,139,711,5,232
...,...,...,...,...,...,...,...
95,96,YT.X,Cherry Token,2,6,95,3
96,97,AMM.X,MicroMoney,2,8,116,4
97,98,ZERO.X,Zero Exchange,2,6,83,3
98,99,CAD.X,Candy Protocol,2,0,143,2


In [115]:
top_5_trending_crypto = list(crypto.sort_values("rank")[:5]["ticker"])

top_5_trending_crypto = [x.replace(".X", "") for x in top_5_trending_crypto]
top_5_trending_crypto = ['BTC', 'ETH', 'UST', 'LUNA', 'USDT']

In [119]:
crypto_prices_df = alpaca.get_bars(top_5_trending_crypto, timeframe = timeframe, start = start_date).df

crypto_prices_df = crypto_prices_df.loc[crypto_prices_df['symbol'].isin(top_5_trending_crypto)][["close", "symbol"]]

crypto_prices_top5 = crypto_prices_df.pivot_table(columns="symbol", index="timestamp")

crypto_prices_top5
# crypto_prices_df["symbol"].unique()

close         
symbol                       LUNA      UST
timestamp                                 
2022-05-16 08:00:00+00:00  5.7600      NaN
2022-05-16 11:00:00+00:00  5.9100      NaN
2022-05-16 11:30:00+00:00  5.8200      NaN
2022-05-16 12:00:00+00:00  6.0500      NaN
2022-05-16 12:30:00+00:00  5.9000      NaN
2022-05-16 13:00:00+00:00  6.3100      NaN
2022-05-16 13:30:00+00:00  5.8250  54.6200
2022-05-16 14:00:00+00:00  5.8750      NaN
2022-05-16 14:30:00+00:00  5.7200      NaN
2022-05-16 15:00:00+00:00  5.6501      NaN
2022-05-16 15:30:00+00:00  5.6550      NaN
2022-05-16 16:00:00+00:00  5.5750      NaN
2022-05-16 16:30:00+00:00  5.5300      NaN
2022-05-16 17:00:00+00:00  5.5700  54.8400
2022-05-16 17:30:00+00:00  5.6400  54.7599
2022-05-16 18:00:00+00:00  5.7450  54.7000
2022-05-16 18:30:00+00:00  5.7159      NaN
2022-05-16 19:00:00+00:00  5.7100  54.7600
2022-05-16 19:30:00+00:00  5.6300      NaN
2022-05-16 20:00:00+00:00  5.6800      NaN
2022-05-16 20:30:00+00:00  5.5100      NaN
2022-05-16 21:00:00+00:00  5.7500      NaN
2022-05-16 21:30:00+00:00  5.7200      NaN
2022-05-17 12:30:00+00:00  5.7000      NaN
2022-05-17 13:30:00+00:00  5.8100  54.0100
2022-05-17 14:00:00+00:00  5.9700  54.0100
2022-05-17 14:30:00+00:00  5.9300      NaN
2022-05-17 15:00:00+00:00  6.0100      NaN
2022-05-17 15:30:00+00:00  6.0279  54.0500
2022-05-17 16:00:00+00:00  6.1399  53.9900
2022-05-17 16:30:00+00:00  6.1400      NaN
2022-05-17 17:00:00+00:00  6.1800      NaN
2022-05-17 17:30:00+00:00  6.1000      NaN
2022-05-17 18:00:00+00:00  6.1300  54.0407
2022-05-17 18:30:00+00:00  6.1389      NaN
2022-05-17 19:00:00+00:00  6.1200      NaN
2022-05-17 19:30:00+00:00  6.1400      NaN
2022-05-17 20:00:00+00:00  6.1400      NaN